## Setting working directory

In [1]:
import os

# Path to directory where you want working directory #
path = 'C:/Users/Rune-/OneDrive - Aarhus universitet/Oecon/8. semester/Info 256/Projekt/NeuralCoref'

# Setting the working directory
os.chdir(path)

# Checking the working directory
os.getcwd()

'C:\\Users\\Rune-\\OneDrive - Aarhus universitet\\Oecon\\8. semester\\Info 256\\Projekt\\NeuralCoref'

## Importing packages

In [2]:
import pandas as pd
import numpy as np
from math import log
from calculatescore import eval_gender
from sklearn.metrics import f1_score, recall_score, precision_score

## Loading data 

In [5]:
df = pd.read_csv('data/gendered-pronoun-resolution/test_stage_1.tsv', sep = '\t', index_col = 0)
df.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
ID,,,,,,,,
development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,Pauline,207,http://en.wikipedia.org/wiki/List_of_Teachers_...
development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,Rivera,294,http://en.wikipedia.org/wiki/Jessica_Rivera


## Creating male and female dataset

In [6]:
set(df['Pronoun'])

{'He', 'Her', 'His', 'She', 'he', 'her', 'him', 'his', 'she'}

In [7]:
# Male pronouns
male_pn = ['He', 'His', 'he', 'him', 'his']

# Female pronouns
female_pn = ['Her', 'She', 'her', 'she']

In [8]:
df_male = []

df_female = []

for i in range(df.shape[0]):
    pn = df['Pronoun'][i]
    if pn in male_pn:
        df_male.append(df.iloc[i, :])
    else:
        df_female.append(df.iloc[i, :])

df_male = pd.DataFrame(df_male)
df_male.index.name = 'ID'

df_female = pd.DataFrame(df_female)
df_female.index.name = 'ID'

In [9]:
print('Shape of male dataframe: (rows = %s, cols = %s)' % df_male.shape)
print('Shape of female dataframe: (rows = %s, cols = %s)' % df_female.shape)

Shape of male dataframe: (rows = 1000, cols = 8)
Shape of female dataframe: (rows = 1000, cols = 8)


In [10]:
df_male.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,URL
ID,,,,,,,,
development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,Bernard Leach,251,http://en.wikipedia.org/wiki/Warren_MacKenzie
development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,De la Sota,246,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
development-4,The current members of Crime have also perform...,his,321,Hell,174,Henry Rosenthal,336,http://en.wikipedia.org/wiki/Crime_(band)
development-7,Reb Chaim Yaakov's wife is the sister of Rabbi...,his,273,Reb Asher,152,Akiva Eiger,253,http://en.wikipedia.org/wiki/Asher_Arieli
development-8,Slant Magazine's Sal Cinquemani viewed the alb...,his,337,Greg Kot,173,Robert Christgau,377,http://en.wikipedia.org/wiki/The_Truth_About_L...


### Find right answers for each gender and create dataset with right answers

In [11]:
df_ans = pd.read_csv('data/right_answers.csv', sep = ',', index_col = 0)
df_ans.head()

,A,B,NEITHER
ID,,,
development-1,1,0,0
development-2,1,0,0
development-3,0,1,0
development-4,0,1,0
development-5,0,1,0


In [12]:
df_male_ans = df_ans.loc[list(df_male.index), :]
df_male_ans.to_csv('data/male_ans.csv')

df_female_ans = df_ans.loc[list(df_female.index), :]
df_female_ans.to_csv('data/female_ans.csv')

## Generate same vote datasets

In [38]:
# Always predict A
data = np.zeros((1000,3),dtype=int)
data[:,0] = np.ones((1000),dtype=int) # always guess on A
predictions = pd.DataFrame(data, columns=['A','B','NEITHER'])
predictions.to_csv('data/pred_A_half.csv')

# Always predict B
data = np.zeros((1000,3),dtype=int)
data[:,1] = np.ones((1000),dtype=int) # always guess on A
predictions = pd.DataFrame(data, columns=['A','B','NEITHER'])
predictions.to_csv('data/pred_B_half.csv')

# Always predict Neither 
data = np.zeros((1000,3),dtype=int)
data[:,2] = np.ones((1000),dtype=int) # always guess on A
predictions = pd.DataFrame(data, columns=['A','B','NEITHER'])
predictions.to_csv('data/pred_N_half.csv')

## Calculate precision, recall and F1 scores

### Only A answers

In [39]:
# Male #
precision_m, recall_m, f1_m = evaluate('data/male_ans.csv', 'data/pred_A_half.csv')
# Run twice to avoid error
print('Male precision: A = %s, B = %s, Neither = %s' % tuple(precision_m.round(4)))
print('Male recall: A = %s, B = %s, Neither = %s' % tuple(recall_m))
print('Male F1: A = %s, B = %s, Neither = %s \n' % tuple(f1_m.round(4)))

# Female #
precision_fm, recall_fm, f1_fm = evaluate('data/female_ans.csv', 'data/pred_A_half.csv')
print('Female precision: A = %s, B = %s, Neither = %s' % tuple(precision_fm))
print('Female recall: A = %s, B = %s, Neither = %s' % tuple(recall_fm))
print('Female F1: A = %s, B = %s, Neither = %s' % tuple(f1_fm.round(4)))

Male precision: A = 0.438, B = 0.0, Neither = 0.0
Male recall: A = 1.0, B = 0.0, Neither = 0.0
Male F1: A = 0.6092, B = 0.0, Neither = 0.0 

Female precision: A = 0.436, B = 0.0, Neither = 0.0
Female recall: A = 1.0, B = 0.0, Neither = 0.0
Female F1: A = 0.6072, B = 0.0, Neither = 0.0


### Only B answers

In [42]:
# Male #
precision_m, recall_m, f1_m = evaluate('data/male_ans.csv', 'data/pred_B_half.csv')
# Run twice to avoid error
print('Male precision: A = %s, B = %s, Neither = %s' % tuple(precision_m.round(4)))
print('Male recall: A = %s, B = %s, Neither = %s' % tuple(recall_m))
print('Male F1: A = %s, B = %s, Neither = %s \n' % tuple(f1_m.round(4)))

# Female #
precision_fm, recall_fm, f1_fm = evaluate('data/female_ans.csv', 'data/pred_B_half.csv')
print('Female precision: A = %s, B = %s, Neither = %s' % tuple(precision_fm))
print('Female recall: A = %s, B = %s, Neither = %s' % tuple(recall_fm))
print('Female F1: A = %s, B = %s, Neither = %s' % tuple(f1_fm.round(4)))

Male precision: A = 0.0, B = 0.459, Neither = 0.0
Male recall: A = 0.0, B = 1.0, Neither = 0.0
Male F1: A = 0.0, B = 0.6292, Neither = 0.0 

Female precision: A = 0.0, B = 0.466, Neither = 0.0
Female recall: A = 0.0, B = 1.0, Neither = 0.0
Female F1: A = 0.0, B = 0.6357, Neither = 0.0


### Only Neither answers

In [43]:
# Male #
precision_m, recall_m, f1_m = evaluate('data/male_ans.csv', 'data/pred_N_half.csv')
# Run twice to avoid error
print('Male precision: A = %s, B = %s, Neither = %s' % tuple(precision_m.round(4)))
print('Male recall: A = %s, B = %s, Neither = %s' % tuple(recall_m))
print('Male F1: A = %s, B = %s, Neither = %s \n' % tuple(f1_m.round(4)))

# Female #
precision_fm, recall_fm, f1_fm = evaluate('data/female_ans.csv', 'data/pred_N_half.csv')
print('Female precision: A = %s, B = %s, Neither = %s' % tuple(precision_fm))
print('Female recall: A = %s, B = %s, Neither = %s' % tuple(recall_fm))
print('Female F1: A = %s, B = %s, Neither = %s' % tuple(f1_fm.round(4)))

Male precision: A = 0.0, B = 0.0, Neither = 0.103
Male recall: A = 0.0, B = 0.0, Neither = 1.0
Male F1: A = 0.0, B = 0.0, Neither = 0.1868 

Female precision: A = 0.0, B = 0.0, Neither = 0.098
Female recall: A = 0.0, B = 0.0, Neither = 1.0
Female F1: A = 0.0, B = 0.0, Neither = 0.1785


## Using the made function

In [3]:
# Gold path
path_gold = 'data/right_answers.csv'

# Path full data #
path_full = 'data/gendered-pronoun-resolution/test_stage_1.tsv'
df = pd.read_csv(path_full, sep = '\t', index_col = 0) 

# Create predictions #
# Always predict A
data = np.zeros((2000,3),dtype=int)
data[:,0] = np.ones((2000),dtype=int) # always guess on A
predictions = pd.DataFrame(data, columns=['A','B','NEITHER'], index = list(df.index))
predictions.to_csv('data/pred_one_class.csv')
path_pred = 'data/pred_one_class.csv'


In [5]:
male_sc, female_sc = eval_gender(path_gold, path_pred, path_full)

Male precision: A = 0.438, B = 0.0, Neither = 0.0
Male recall: A = 1.0, B = 0.0, Neither = 0.0
Male F1: A = 0.6092, B = 0.0, Neither = 0.0 

Female precision: A = 0.436, B = 0.0, Neither = 0.0
Female recall: A = 1.0, B = 0.0, Neither = 0.0
Female F1: A = 0.6072, B = 0.0, Neither = 0.0
